# Clustering - Treinar K-means - Economiza+ MVP

**Objetivo:** Treinar modelo K-means para segmentação de usuários

**Checklist:**
- Normalizar features (StandardScaler)
- Treinar K-means para K=3, 4, 5
- Elbow Curve
- Escolher melhor K
- Salvar modelos

## 1. Imports e Configurações

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import joblib
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

In [ ]:
# Teste rápido - verificar se sklearn está instalado
import sys
print(f"Python: {sys.version}")
print(f"Caminho: {sys.executable}")

# Testar import básico
try:
    import sklearn
    print(f"✓ scikit-learn {sklearn.__version__} instalado!")
except ImportError as e:
    print(f"✗ Erro: {e}")

Python: 3.12.3 (main, Jun 22 2025, 11:00:19) [GCC 13.3.0]
Caminho: /home/celina/.pyenv/versions/economiza-ia-env/bin/python
✓ scikit-learn 1.8.0 instalado!


: 

## 2. Carregar Features

In [1]:
# Carregar features processadas
df = pd.read_csv('../data/processed/features_clustering.csv')
print(f"Shape: {df.shape}")
df.head()

NameError: name 'pd' is not defined

In [ ]:
# Verificar features disponíveis
print("Colunas disponíveis:")
print(df.columns.tolist())
print(f"\nValores nulos: {df.isnull().sum().sum()}")

## 3. Normalização com StandardScaler

In [ ]:
# Separar ID e features
id_col = 'usuario_id' if 'usuario_id' in df.columns else df.columns[0]
user_ids = df[id_col]
features = df.drop(columns=[id_col])

print(f"Features para clustering: {features.shape[1]} colunas")
print(f"Usuários: {features.shape[0]}")

In [ ]:
# Normalizar features
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

print("Normalização concluída!")
print(f"Shape: {features_normalized.shape}")
print(f"Média: {features_normalized.mean():.4f}")
print(f"Desvio padrão: {features_normalized.std():.4f}")

## 4. Treinar K-means para K=3, 4, 5

In [ ]:
# Treinar modelos para diferentes valores de K
K_values = [3, 4, 5]
models = {}
inertias = []

for k in K_values:
    print(f"\nTreinando K-means com K={k}...")
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    kmeans.fit(features_normalized)
    
    models[k] = kmeans
    inertias.append(kmeans.inertia_)
    
    print(f"  Inércia: {kmeans.inertia_:.2f}")
    print(f"  Iterações: {kmeans.n_iter_}")
    
    # Distribuição dos clusters
    unique, counts = np.unique(kmeans.labels_, return_counts=True)
    print(f"  Distribuição: {dict(zip(unique, counts))}")

## 5. Elbow Curve

In [ ]:
# Plotar Elbow Curve
plt.figure(figsize=(10, 6))
plt.plot(K_values, inertias, 'bo-', linewidth=2, markersize=10)
plt.xlabel('Número de Clusters (K)', fontsize=12)
plt.ylabel('Inércia (Within-Cluster Sum of Squares)', fontsize=12)
plt.title('Elbow Curve - Escolha do K Ótimo', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(K_values)

# Adicionar valores no gráfico
for k, inertia in zip(K_values, inertias):
    plt.annotate(f'{inertia:.1f}', 
                xy=(k, inertia), 
                xytext=(0, 10),
                textcoords='offset points',
                ha='center',
                fontsize=10)

plt.tight_layout()
plt.savefig('../outputs/elbow_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nElbow Curve salva em: outputs/elbow_curve.png")

## 6. Escolher Melhor K

In [ ]:
# Calcular taxa de redução da inércia
print("Análise da redução de inércia:")
print("="*50)
for i in range(len(K_values)-1):
    k_atual = K_values[i]
    k_prox = K_values[i+1]
    reducao = inertias[i] - inertias[i+1]
    percentual = (reducao / inertias[i]) * 100
    print(f"K={k_atual} → K={k_prox}: {reducao:.2f} ({percentual:.1f}% de redução)")

# Escolher K (baseado no cotovelo visual e redução significativa)
best_k = 4  # Ajustar conforme análise visual
print(f"\n{'='*50}")
print(f"✓ K escolhido: {best_k}")
print(f"{'='*50}")

## 7. Analisar Clusters do Modelo Escolhido

In [ ]:
# Obter labels do melhor modelo
best_model = models[best_k]
cluster_labels = best_model.labels_

# Adicionar labels ao dataframe original
df_clustered = df.copy()
df_clustered['cluster'] = cluster_labels

# Estatísticas por cluster
print(f"\nDistribuição dos {best_k} clusters:")
print(df_clustered['cluster'].value_counts().sort_index())
print(f"\nPercentuais:")
print((df_clustered['cluster'].value_counts(normalize=True) * 100).sort_index().round(2))

In [ ]:
# Perfil dos clusters (estatísticas das features originais)
cluster_profiles = df_clustered.groupby('cluster')[features.columns].mean()
print("\nPerfil médio dos clusters:")
cluster_profiles

In [ ]:
# Visualizar distribuição dos clusters (primeiras 2 features principais)
plt.figure(figsize=(12, 5))

# Scatter plot
plt.subplot(1, 2, 1)
scatter = plt.scatter(features_normalized[:, 0], 
                     features_normalized[:, 1], 
                     c=cluster_labels, 
                     cmap='viridis', 
                     alpha=0.6,
                     edgecolors='black',
                     linewidth=0.5)
plt.xlabel('Feature 1 (normalizada)', fontsize=11)
plt.ylabel('Feature 2 (normalizada)', fontsize=11)
plt.title('Visualização dos Clusters', fontsize=12, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)

# Tamanho dos clusters
plt.subplot(1, 2, 2)
cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()
colors = plt.cm.viridis(np.linspace(0, 1, len(cluster_counts)))
bars = plt.bar(cluster_counts.index, cluster_counts.values, color=colors, edgecolor='black')
plt.xlabel('Cluster', fontsize=11)
plt.ylabel('Número de Usuários', fontsize=11)
plt.title('Distribuição dos Clusters', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

# Adicionar valores nas barras
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('../outputs/cluster_visualization.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nVisualização salva em: outputs/cluster_visualization.png")

## 8. Salvar Modelos e Artefatos

In [ ]:
# Criar diretório se não existir
import os
os.makedirs('../models', exist_ok=True)

# Salvar scaler
joblib.dump(scaler, '../models/scaler.pkl')
print("✓ Scaler salvo em: models/scaler.pkl")

# Salvar todos os modelos K-means
for k, model in models.items():
    filename = f'../models/kmeans_k{k}.pkl'
    joblib.dump(model, filename)
    print(f"✓ Modelo K={k} salvo em: models/kmeans_k{k}.pkl")

# Salvar modelo escolhido com nome especial
joblib.dump(best_model, '../models/kmeans_best.pkl')
print(f"\n✓ Modelo final (K={best_k}) salvo em: models/kmeans_best.pkl")

In [ ]:
# Salvar dataframe com clusters
df_clustered.to_csv('../data/processed/usuarios_clustered.csv', index=False)
print("✓ Dados com clusters salvos em: data/processed/usuarios_clustered.csv")

## 9. Resumo Final

In [ ]:
print("="*60)
print("RESUMO DO CLUSTERING")
print("="*60)
print(f"\n✓ Modelos treinados: K={K_values}")
print(f"✓ Modelo escolhido: K={best_k}")
print(f"✓ Total de usuários: {len(df)}")
print(f"✓ Features utilizadas: {features.shape[1]}")
print(f"\nInércias:")
for k, inertia in zip(K_values, inertias):
    marker = "👉" if k == best_k else "  "
    print(f"{marker} K={k}: {inertia:.2f}")
print(f"\n✓ Artefatos salvos:")
print(f"  - models/scaler.pkl")
print(f"  - models/kmeans_k3.pkl, kmeans_k4.pkl, kmeans_k5.pkl")
print(f"  - models/kmeans_best.pkl (K={best_k})")
print(f"  - data/processed/usuarios_clustered.csv")
print(f"  - outputs/elbow_curve.png")
print(f"  - outputs/cluster_visualization.png")
print("\n" + "="*60)
print("CLUSTERING CONCLUÍDO! ✨")
print("="*60)